### Feed Forward

In [465]:
%run core.ipynb

In [463]:
# %load feedforward..py
import np as _np
import numpy as np


def feedforward(a, N, x):
    for W, B in N:
        x = a(np.dot(W, x) + B)
    return x


def get_random_weights_and_biases(k, l, m=None, layers=0):
    N = [(np.random_matrix(l, k), np.random_vector(l))]
    for _ in range(layers):
        N.append((np.random_matrix(l, l), np.random_vector(l)))
    if m is not None:
        N.append((np.random_matrix(m, l), np.random_vector(m)))
    return N


class FeedForward:

    def __init__(self, dim, layers, a):
        self.N = get_random_weights_and_biases(*dim, layers)
        self.a = a

    def __call__(self, x):
        return self.feedforward(x)

    def __repr__(self):
        return repr(self.N)

    def feedforward(self, x):
        return feedforward(self.a, self.N, x)


class FeedForwardSingleLayer(FeedForward):

    def __init__(self, dim, a, da):
        super().__init__(dim=(*dim, None), layers=0, a=a)
        self.da = da

    def train(self, T, s=0.01, epochs=50):
        for _ in range(epochs):
            for x, y in T:
                z = feedforward(np.linear, self.N, x)
                az, daz = self.a(z), self.da(z)
                d = y - az
                for W, B in self.N:
                    for i, (w, b) in enumerate(zip(W, B)):
                        w += s * d[i] * daz[i] * x
                        b += s * d[i] * daz[i]


In [380]:
a = lambda x: np.multiply(x, 1)
N = [
    (np.matrix((2, 2), 
               (1, -1),
               (2, 2)),  
     np.vector(1, -1, 0)),
    (np.matrix((1, 2, 2),
               (0, 0, 3),
               (1, 2, 2)), 
     np.vector(1, -1, -2)),
    (np.matrix((2, 2, 1),), 
     np.vector(1,)),
]
x = np.vector(1, -1)

feedforward(a, N, x)

array([8])

In [381]:
a = np.sigmoid
N = get_random_weights_and_biases(2, 3, 2, layers=3)
x = np.random_vector(2)
feedforward(a, N, x)


array([0.84968362, 0.80859287])

In [382]:
f = FeedForward(dim=(2, 3, 2), layers=3, a=np.sigmoid)
x = np.random_vector(2)
f(x)

array([0.9247423, 0.84952  ])

In [383]:
f = FeedForwardSingleLayer(dim=(3, 2), a=np.sigmoid, da=np.dx_sigmoid)
f

[(array([[0.68308225, 0.78703011, 0.23961784],
       [0.79022633, 0.60294131, 0.12961675]]), array([0.04342562, 0.03300063]))]

In [485]:
from dec2bin import int2dec, int2bin, bin2int


class Dec2Bin(FeedForwardSingleLayer):

    def __init__(self, a=np.sigmoid, da=np.dx_sigmoid, 
                 classify=lambda x: heaviside(x, offset=0.5), s=0.01, epochs=100):
        super().__init__(dim=(10, 4), a=a, da=da)
        self.classify = classify
        self.train([(np.vector(*int2dec(n)), np.vector(*int2bin(n))) for n in range(10)], s, epochs)

    def __call__(self, n):
        return tuple([self.classify(t) for t in super().__call__(int2dec(n))])

    
def check_dec2bin_feedforward(f):
    for n in range(10):
        y = f(n)
        z = tuple(int2bin(n))
        if y == z:
            print(f"{n} = {y} {_ok}")
        else:
            print(f"{n} = {z} ≠ {_red(y)} {_nok}")
            

In [486]:
a, da, classify = np.sigmoid, np.dx_sigmoid, lambda x: heaviside(x, offset=0.5)
s, epochs = 0.01, 1500
f = Dec2Bin(a=a, da=da, classify=classify, s=s, epochs=epochs)

check_dec2bin_feedforward(f)

0 = (0, 0, 0, 0) ✔
1 = (1, 0, 0, 0) ✔
2 = (0, 1, 0, 0) ✔
3 = (1, 1, 0, 0) ✔
4 = (0, 0, 1, 0) ✔
5 = (1, 0, 1, 0) ✔
6 = (0, 1, 1, 0) ✔
7 = (1, 1, 1, 0) ✔
8 = (0, 0, 0, 1) ✔
9 = (1, 0, 0, 1) ✔


In [487]:
a, da, classify = np.relu, np.dx_relu, lambda x: heaviside(x, offset=0.5)
s, epochs = 0.01, 150
f = Dec2Bin(a=a, da=da, classify=classify, s=s, epochs=epochs)

check_dec2bin_feedforward(f)

0 = (0, 0, 0, 0) ✔
1 = (1, 0, 0, 0) ✔
2 = (0, 1, 0, 0) ✔
3 = (1, 1, 0, 0) ✔
4 = (0, 0, 1, 0) ✔
5 = (1, 0, 1, 0) ✔
6 = (0, 1, 1, 0) ✔
7 = (1, 1, 1, 0) ✔
8 = (0, 0, 0, 1) ✔
9 = (1, 0, 0, 1) ✔


In [492]:
class BooleanOperator(FeedForwardSingleLayer):

    def __init__(self, operator, a=np.sigmoid, da=np.dx_sigmoid, 
                 classify=lambda x: heaviside(x, offset=0.5), s=0.01, epochs=50):
        super().__init__(dim=(2, 1), a=a, da=da)
        self.classify = classify
        self.operator = operator
        self.train([(np.vector(*t), np.vector(y)) for t, y in BOOLEAN_TESTDATA[operator]], s, epochs)

    def __call__(self, x, y):
        return [self.classify(t) for t in super().__call__(np.vector(x, y))][0]

    
def check_boolean_feedforward(f):
    for t, y in BOOLEAN_TESTDATA[f.operator]:
        z = f(*t)
        if y == z:
            print(f"and{t} = {z} {_ok}")
        else:
            print(f"and{t} = {y} ≠ {z} {_nok}")
            

In [493]:
a, da, classify = np.relu, np.dx_relu, lambda x: heaviside(x, offset=0.5)
s, epochs = 0.01, 500
f = BooleanOperator(AND, a=a, da=da, classify=classify, s=s, epochs=epochs)

check_boolean_feedforward(f)

and(0, 0) = 0 ✔
and(0, 1) = 0 ✔
and(1, 0) = 0 ✔
and(1, 1) = 1 ✔
